In [ ]:
from tapo_plug import tapoPlugApi
import bluepy.btle as btle
import datetime
import argparse
import os

### For bluepy Peripheral class
global dataByte
dataByte = 0
class ReadDelegate(btle.DefaultDelegate):
    def handleNotification(self, cHandle, data):
        # Access global variables insid class
        global dataByte
        dataByte = data

blue_address = "FC:45:C3:DC:99:87"

### Connect to HC-08 bluetooth Module
p = 0
while p == 0:
    try:
        if p != 0:
            continue
        p = btle.Peripheral(blue_address)
        p.withDelegate(ReadDelegate())
    except:
        pass

while True:
    try:
        ### Message 1
        p.waitForNotifications(0)
        # Change byte to int data
        light = ""
        for i in range(len(dataByte)):
            d = str(dataByte[i])
            light = light + d
        light = int(light)
        print("light : ", light)

        ### Save data into file
        with open('light_intensity.txt', 'w') as f:
            f.write(light)

        ### Control  
        if round(light) < 200:
            device = {"tapoIp": "192.168.XX.XXX", "tapoEmail": "ntubblab@gmail.com", "tapoPassword": "ntubime405"}
            response = tapoPlugApi.plugOn(device)
            print(response)
        if round(light) > 500:
            device = {"tapoIp": "192.168.XX.XXX", "tapoEmail": "ntubblab@gmail.com", "tapoPassword": "ntubime405"}
            response = tapoPlugApi.plugOff(device)
            print(response)
            
    ### Reconnect
    except Exception as e:
        print(e)
        try:
            p = 0
            while p == 0:
                if p != 0:
                    continue
                p = btle.Peripheral(blue_address)
                p.withDelegate(ReadDelegate())
                # Initialize
                light = None
        except:
            pass